This notebook looks to replicate the tutorial material from databrick's own tutorials, only using open source libraries and tools.  
  
Below is a list of the tools used.

http://192.168.1.123:8998  
Livy Server: http://192.168.1.123:8998/ui  
Spark Master: http://192.168.1.123:8080/  
Spark Magic: https://github.com/jupyter-incubator/sparkmagic/blob/master/examples/Magics%20in%20IPython%20Kernel.ipynb  

Let's load up sparkmagic to be able to communicate without spark cluster

In [1]:
%load_ext sparkmagic.magics

Now let's configure our connection

In [2]:
%manage_spark

TWFnaWNzQ29udHJvbGxlcldpZGdldChjaGlsZHJlbj0oVGFiKGNoaWxkcmVuPShNYW5hZ2VTZXNzaW9uV2lkZ2V0KGNoaWxkcmVuPShIVE1MKHZhbHVlPXUnPGJyLz4nKSwgSFRNTCh2YWx1ZT3igKY=


Added endpoint http://192.168.1.123:8998/
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,None,spark,idle,,,✔


SparkSession available as 'spark'.


Let's read the data used in the tutorial

In [3]:
%%spark
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

val marketsFile = sc.textFile("file:///home/churtado/notebooks/data/farmers_market.csv")

val taxes2013 = spark.read.format("csv").option("header", "true").load("file:///home/churtado/notebooks/data/zipcodes.csv")
taxes2013.createOrReplaceTempView("taxes2013")
val markets = spark.read.format("csv").option("header", "true").load("file:///home/churtado/notebooks/data/farmers_market.csv")
markets.createOrReplaceTempView("markets")

sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@1a54232e
marketsFile: org.apache.spark.rdd.RDD[String] = file:///home/churtado/notebooks/data/farmers_market.csv MapPartitionsRDD[1] at textFile at <console>:26
taxes2013: org.apache.spark.sql.DataFrame = [STATEFIPS: string, STATE: string ... 112 more fields]
markets: org.apache.spark.sql.DataFrame = [FMID: string, MarketName: string ... 57 more fields]


We can use sql magic to view the data

In [4]:
%%spark -c sql
SHOW TABLES

,database,tableName,isTemporary
0,default,cleaned_taxes,False
1,default,cleaned_taxes_state,False
2,default,income_state,False
3,,markets,True
4,,taxes2013,True


In [5]:
%%spark -c sql -q -o df_markets 
SELECT * FROM markets

We can now use pandas dataframes for the data

In [6]:
df_markets.head(3)

,FMID,MarketName,Website,Facebook,city,County,State,zip,Season1Date,Season1Time,...,Tofu,WildHarvested,updateTime,street,OtherMedia,Location,Twitter,Youtube,Season3Date,Season3Time
0,1018261,Caledonia Farmers Market Association - Danville,https://sites.google.com/site/caledoniafarmers...,https://www.facebook.com/Danville.VT.Farmers.M...,Danville,Caledonia,Vermont,05828,06/14/2017 to 08/30/2017,Wed: 9:00 AM-1:00 PM;,...,N,N,2017-06-20 22:43:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1018318,Stearns Homestead Farmers' Market,http://www.StearnsHomestead.com,StearnsHomesteadFarmersMarket,Parma,Cuyahoga,Ohio,NaN,06/24/2017 to 09/30/2017,Sat: 9:00 AM-1:00 PM;,...,N,N,2017-06-21 17:15:01,6975 Ridge Road,NaN,NaN,NaN,NaN,NaN,NaN
2,1009364,106 S. Main Street Farmers Market,http://thetownofsixmile.wordpress.com/,NaN,Six Mile,NaN,South Carolina,29682,NaN,NaN,...,N,N,2013-01-01 00:00:00,106 S. Main Street,NaN,NaN,NaN,NaN,NaN,NaN


We can visualize the data as well

In [7]:
from autovizwidget.widget.utils import display_dataframe
display_dataframe(df_markets)

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oSFRNTCh2YWx1ZT11J1R5cGU6JyksIEJ1dHRvbihkZXNjcmlwdGlvbj11J1RhYmxlJywgbGF5b3V0PUxheW91dCh3aWR0aD11JzcwcHgnKSzigKY=


Output()

AutoVizWidget()

Now we're going to clean up the data with SQL

In [8]:
%%spark -c sql
DROP TABLE IF EXISTS cleaned_taxes

""


In [9]:
%%spark -c sql -q -o df_cleaned_taxes

CREATE TABLE cleaned_taxes AS
SELECT state, int(zipcode / 10) as zipcode, 
  int(mars1) as single_returns, 
  int(mars2) as joint_returns, 
  int(numdep) as numdep, 
  double(A02650) as total_income_amount,
  double(A00300) as taxable_interest_amount,
  double(a01000) as net_capital_gains,
  double(a00900) as biz_net_income
FROM taxes2013

We want to look at avg income per state

In [10]:
%%spark -c sql
DROP TABLE IF EXISTS income_state

""


In [11]:
%%spark -c sql -q 

CREATE TABLE income_state AS
SELECT state, 
sum(total_income_amount) / sum(1) as total_income
FROM cleaned_taxes
group by state

In [12]:
%%spark -c sql -q  -o df_income_state

select * from income_state

Let's try to plot income by state using plotly

In [13]:
import plotly
plotly.tools.set_credentials_file(username='churtado', api_key='iaMRV6ydU9Ove5Yfy0R7')

In [14]:
states = df_income_state["state"]
values = df_income_state["total_income"]

import plotly.plotly as py
import pandas as pd

data = [ dict(
        type='choropleth',
        autocolorscale = True,
        locations = df_income_state['state'],
        z = df_income_state['total_income'].astype(float),
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Total Avg Income")
        ) ]

layout = dict(
        title = 'Map of avg income',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

In [15]:
%%spark -c sql 

SELECT * FROM cleaned_taxes limit 5

,state,zipcode,single_returns,joint_returns,numdep,total_income_amount,taxable_interest_amount,net_capital_gains,biz_net_income
0,TX,7924,70,90,130,6226.0,25.0,0.0,273.0
1,TX,7924,0,60,60,4063.0,52.0,168.0,133.0
2,TX,7924,0,30,40,3347.0,0.0,0.0,0.0
3,TX,7924,0,30,0,5660.0,40.0,0.0,0.0
4,TX,7924,0,0,0,0.0,0.0,0.0,0.0


In [16]:
%%spark -c sql -q -o df_capital_gains

SELECT zipcode AS zipcode, SUM(net_capital_gains) AS cap_gains
FROM cleaned_taxes 
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
GROUP BY zipcode
ORDER BY cap_gains ASC
LIMIT 10

Let's look at the set of zip codes with the lowest total capital gains and plot the results.

In [17]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=df_capital_gains["zipcode"],
    y=df_capital_gains["cap_gains"]
)
data = [trace1]
layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        type='category',
        zeroline=False,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=1,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        #linewidth=6
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=1,
        zerolinecolor='#969696',
        zerolinewidth=1,
        linecolor='#636363',
        #linewidth=6
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='axes-lines')

Let's look at a combination of capital gains and business net income to see what we find.

In [18]:
%%spark -c sql -q -o df_capital_and_business_income

SELECT zipcode, 
  SUM(biz_net_income) as business_net_income, 
  SUM(net_capital_gains) as capital_gains, 
  SUM(net_capital_gains) + SUM(biz_net_income) as capital_and_business_income
FROM cleaned_taxes 
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
GROUP BY zipcode
ORDER BY capital_and_business_income DESC
LIMIT 50

In [21]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=df_capital_and_business_income["zipcode"],
    y=df_capital_and_business_income["business_net_income"],
    name='Business Net Income'
)

trace2 = go.Bar(
    x=df_capital_and_business_income["zipcode"],
    y=df_capital_and_business_income["capital_gains"],
    name='Capital Gains'
)

trace3 = go.Bar(
    x=df_capital_and_business_income["zipcode"],
    y=df_capital_and_business_income["capital_and_business_income"],
    name='Capital and Business Income'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='group',
    xaxis=dict(
        showgrid=False,
        type='category',
        zeroline=False,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=1,
        zerolinecolor='#969696',
        zerolinewidth=4,
        linecolor='#636363',
        #linewidth=6
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=True,
        mirror='ticks',
        gridcolor='#bdbdbd',
        gridwidth=1,
        zerolinecolor='#969696',
        zerolinewidth=1,
        linecolor='#636363',
        #linewidth=6
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='kpis')

If we want to see how a query will run we can look at the plan

In [56]:
%%spark -c sql -q -o df_explain_plan
EXPLAIN 
  SELECT zipcode, 
    SUM(biz_net_income) as net_income, 
    SUM(net_capital_gains) as cap_gains, 
    SUM(net_capital_gains) + SUM(biz_net_income) as combo
  FROM cleaned_taxes 
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
  GROUP BY zipcode
  ORDER BY combo desc
  limit 50

In [55]:
import sys

pd.set_option("display.max_colwidth", 10000)
plan = df_explain_plan["plan"].to_string().replace('\\n', '\n')
sys.stdout.write(plan)

0    == Physical Plan ==
TakeOrderedAndProject(limit=50, orderBy=[combo#809 DESC NULLS LAST], output=[zipcode#813,net_income#807,cap_gains#808,combo#809])
+- *(2) HashAggregate(keys=[zipcode#813], functions=[sum(biz_net_income#820), sum(net_capital_gains#819)])
   +- Exchange hashpartitioning(zipcode#813, 200)
      +- *(1) HashAggregate(keys=[zipcode#813], functions=[partial_sum(biz_net_income#820), partial_sum(net_capital_gains#819)])
         +- *(1) Filter ((isnotnull(zipcode#813) && NOT (zipcode#813 = 0)) && NOT (zipcode#813 = 9999))
            +- HiveTableScan [zipcode#813, net_capital_gains#819, biz_net_income#820], HiveTableRelation `default`.`cleaned_taxes`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, [state#812, zipcode#813, single_returns#814, joint_returns#815, numdep#816, total_income_amount#817, taxable_interest_amount#818, net_capital_gains#819, biz_net_income#820]

We can cache the table in memory. If we do it with SQL it's done eagerly, and with the API lazily

In [57]:
%%spark
spark.table("cleaned_taxes").cache()

res21: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [state: string, zipcode: int ... 7 more fields]


In [58]:
%%spark -c sql 

CACHE TABLE cleaned_taxes

""


If we query it now, the query will be faster

In [61]:
%%spark -c sql  -q 

SELECT zipcode, 
  SUM(biz_net_income) as net_income, 
  SUM(net_capital_gains) as cap_gains, 
  SUM(net_capital_gains) + SUM(biz_net_income) as combo
FROM cleaned_taxes 
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
GROUP BY zipcode
ORDER BY combo desc
limit 50

This will have a different explain plan

In [62]:
%%spark -c sql -q -o df_explain_plan2
EXPLAIN 
  SELECT zipcode, 
    SUM(biz_net_income) as net_income, 
    SUM(net_capital_gains) as cap_gains, 
    SUM(net_capital_gains) + SUM(biz_net_income) as combo
  FROM cleaned_taxes 
  WHERE NOT (zipcode = 0000 OR zipcode = 9999)
  GROUP BY zipcode
  ORDER BY combo desc
  limit 50

In [63]:
import sys

pd.set_option("display.max_colwidth", 10000)
plan = df_explain_plan2["plan"].to_string().replace('\\n', '\n')
sys.stdout.write(plan)

0    == Physical Plan ==
TakeOrderedAndProject(limit=50, orderBy=[combo#1183 DESC NULLS LAST], output=[zipcode#1188,net_income#1181,cap_gains#1182,combo#1183])
+- *(2) HashAggregate(keys=[zipcode#1188], functions=[sum(biz_net_income#1195), sum(net_capital_gains#1194)])
   +- Exchange hashpartitioning(zipcode#1188, 200)
      +- *(1) HashAggregate(keys=[zipcode#1188], functions=[partial_sum(biz_net_income#1195), partial_sum(net_capital_gains#1194)])
         +- *(1) Filter ((isnotnull(zipcode#1188) && NOT (zipcode#1188 = 0)) && NOT (zipcode#1188 = 9999))
            +- InMemoryTableScan [zipcode#1188, net_capital_gains#1194, biz_net_income#1195], [isnotnull(zipcode#1188), NOT (zipcode#1188 = 0), NOT (zipcode#1188 = 9999)]
                  +- InMemoryRelation [state#1187, zipcode#1188, single_returns#1189, joint_returns#1190, numdep#1191, total_income_amount#1192, taxable_interest_amount#1193, net_capital_gains#1194, biz_net_income#1195], true, 10000, StorageLevel(disk, memory, deserial

Now let's look at the farmer's market data

In [64]:
%%spark -c sql -q
SELECT * FROM MARKETS